In [3]:
import sys
!{sys.executable} -m pip install pip earthengine-api
!{sys.executable} -m pip install pip geemap

  Using cached earthengine_api-0.1.334-py3-none-any.whl
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
  Using cached httplib2-0.21.0-py3-none-any.whl (96 kB)
  Using cached google_api_python_client-2.69.0-py2.py3-none-any.whl (10.7 MB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)
  Using cached geemap-0.19.0-py2.py3-none-any.whl (2.1 MB)
  Using cached geeadd-0.5.6-py3-none-any.whl (30 kB)
  Using cached ee_extra-0.0.14-py3-none-any.whl
  Using cached geocoder-1.38.1-py2.py3-none-any.whl (98 kB)
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  Using cached sankee-0.2.1-py3-none-any.whl
  Using cached eerepr-0.0.4-py3-none-any.whl (9.7 kB)
  Using cached logzero-1.7.0-py2.py3-none-any.whl (16 kB)
  Using cached plotly-5.11.0-py2.py3-none-any.whl (15.3 MB)
  Using cached ratelim-0.1.6-py2.py3-none-any.whl (4.0 kB)


In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
import os, requests, json, geojson, gc
import pandas as pd
import geopandas as gpd
import ee
import geemap

In [5]:
service_account = 'climate-hazard-demo@data-portal-adaptation.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'google_cred.json')
ee.Initialize(credentials)

In [6]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data"

In [7]:
OUTPUT_FILENAME = 'LND-5.csv'

In [8]:
# get list of urbanshift cities
boundary_georef = pd.read_csv('https://cities-urbanshift.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')

# remove cities without tree cover data availability
#tml_not_available_cities = ['BRA-Salvador','MEX-Monterrey']
tml_not_available_cities = []
boundary_georef = boundary_georef[~boundary_georef['geo_name'].isin(tml_not_available_cities)].reset_index(drop=True)
boundary_georef

,geo_name,level,aoi_boundary_name,units_boundary_name,city_name,country_name,country_code,continent
0,ARG-Mendoza,region,ADM3union,ADM3,Mendoza,Argentina,ARG,America
1,ARG-Mar_del_Plata,city,ADM3,ADM4,Mar del Plata city,Argentina,ARG,America
2,ARG-Mar_del_Plata,region,ADM2,NaN,Mar del Plata region,Argentina,ARG,America
3,ARG-Ushuaia,city,ADM4,ADM5,Ushuaia city,Argentina,ARG,America
4,ARG-Ushuaia,region,ADM3,NaN,Ushuaia region,Argentina,ARG,America
5,ARG-Salta,region,ADM2union,ADM3,Salta,Argentina,ARG,America
6,ARG-Buenos_Aires,region,ADM2union,ADM2,Buenos Aires,Argentina,ARG,America
7,BRA-Teresina,city,ADM4union,ADM4,Teresina city,Brazil,BRA,America
8,BRA-Teresina,region,ADM2union,ADM2,Teresina region,Brazil,BRA,America
9,BRA-Florianopolis,city,ADM4union,ADM4,Florianopolis,Brazil,BRA,America


In [23]:
# Convert from GLAD ARD types to simpler habitat types
# ARD legend is at https://storage.googleapis.com/earthenginepartners-hansen/GLCLU2000-2020/legend.xlsx
LANDCLASSES = {}
for j in range(0, 19):
    LANDCLASSES[j] = {'name': 'upland sparse vegetation', 'classval': 1, 'is_habitat': False}
for j in range(19, 25):
    LANDCLASSES[j] = {'name': 'upland short vegetation', 'classval': 2, 'is_habitat': True}
for j in range(25, 49):
    LANDCLASSES[j] = {'name': 'upland tree cover', 'classval': 3, 'is_habitat': True}
for j in range(100, 119):
    LANDCLASSES[j] = {'name': 'wetland sparse vegetation', 'classval': 4, 'is_habitat': True}
for j in range(119, 125):
    LANDCLASSES[j] = {'name': 'wetland short vegetation', 'classval': 5, 'is_habitat': True}
for j in range(125, 149):
    LANDCLASSES[j] = {'name': 'weland tree cover', 'classval': 6, 'is_habitat': True}
for j in range(200, 208):
    LANDCLASSES[j] = {'name': 'open water', 'classval': 7, 'is_habitat': True}
LANDCLASSES[241] = {'name': 'snow and ice', 'classval': 8, 'is_habitat': False}
LANDCLASSES[241] = {'name': 'cropland', 'classval': 9, 'is_habitat': False}
LANDCLASSES[250] = {'name': 'built up', 'classval': 10, 'is_habitat': False}
LANDCLASSES[254] = {'name': 'ocean', 'classval': 11, 'is_habitat': False}

for j in range(256):
    if not j in list(LANDCLASSES.keys()):
        LANDCLASSES[j] = {'name': 'nodata', 'classval': 0, 'is_habitat': False}
        
froms = list(range(256))
tos = [LANDCLASSES[j]['classval'] for j in froms]        
LCLUC2000 = ee.Image('projects/glad/GLCLU2020/LCLUC_2000').remap(froms, tos, 0)
LCLUC2020 = ee.Image('projects/glad/GLCLU2020/LCLUC_2020').remap(froms, tos, 0)

In [24]:
#Convert to habitat/nonhabitat
froms = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
tos = [0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
habitat_2000 = LCLUC2000.remap(froms, tos, 0)
habitat_2020 = LCLUC2020.remap(froms, tos, 0)

In [25]:
new_habitat = habitat_2000.eq(0).multiply(habitat_2020)
new_habitat_types = LCLUC2020.updateMask(new_habitat)

In [26]:
habitat2020_types = LCLUC2020.updateMask(habitat_2020)

In [27]:
def get_ind(fc):
    count_new_habitat_types = new_habitat_types.reduceRegions(**{
        'reducer': ee.Reducer.countDistinctNonNull(),
        'collection': fc,
        'scale': 30
    })
    count_2020_habitat_types = habitat2020_types.reduceRegions(**{
        'reducer': ee.Reducer.countDistinctNonNull(),
        'collection': fc,
        'scale': 30
    })
    ind = (geemap.ee_to_pandas(count_new_habitat_types)['count']) / (geemap.ee_to_pandas(count_2020_habitat_types)['count'])
    return ind

def get_count(fc):
    count_new_habitat_types = new_habitat_types.reduceRegions(**{
        'reducer': ee.Reducer.countDistinctNonNull(),
        'collection': fc,
        'scale': 30
    })
    return (geemap.ee_to_pandas(count_new_habitat_types)['count'])

def get_total(fc):
    count_2020_habitat_types = habitat2020_types.reduceRegions(**{
        'reducer': ee.Reducer.countDistinctNonNull(),
        'collection': fc,
        'scale': 30
    })
    return (geemap.ee_to_pandas(count_2020_habitat_types)['count'])

In [28]:
all_results = []
for i in range(len(boundary_georef)):
    for boundary_name in ['aoi_boundary_name', 'units_boundary_name']:
        if type(boundary_georef.loc[i, boundary_name]) != float: # sometimes boundary_id is nan
            boundary_id = boundary_georef.loc[i, 'geo_name']+'-' + boundary_georef.loc[i, boundary_name]
            print(boundary_id)
            boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-' + boundary_id + '.geojson'
            boundary_path = aws_s3_dir + '/boundaries/v_0/boundary-'+ boundary_id + '.geojson'
            boundary_geo = requests.get(boundary_path).json()
            temp_gdf = gpd.GeoDataFrame.from_features(boundary_geo)
                        
            boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
            temp_gdf = gpd.GeoDataFrame.from_features(boundary_geo)
            #if True or boundary_name == 'aoi_boundary_name':  # Calculates naturalarea for aoi, uses it for all unit-of-analysis calculations.
               
            temp_gdf['LND-5'] = get_ind(boundary_geo_ee).fillna(-9999)
            all_results.append(temp_gdf.copy())
            outp = pd.concat(all_results, axis=0)[['geo_id', 'geo_level', 'geo_name', 'geo_parent_name', 'LND-5']]
            outp.to_csv(OUTPUT_FILENAME)
    

ARG-Mendoza-ADM3union
ARG-Mar_del_Plata-ADM3
ARG-Mar_del_Plata-ADM4
ARG-Mar_del_Plata-ADM2
ARG-Ushuaia-ADM4
ARG-Ushuaia-ADM5
ARG-Ushuaia-ADM3
ARG-Salta-ADM2union
ARG-Salta-ADM3
ARG-Buenos_Aires-ADM2union
ARG-Buenos_Aires-ADM2
BRA-Teresina-ADM4union
BRA-Teresina-ADM4
BRA-Teresina-ADM2union
BRA-Teresina-ADM2
BRA-Florianopolis-ADM4union
BRA-Florianopolis-ADM4
BRA-Florianopolis-ADM2union
BRA-Florianopolis-ADM2
BRA-Belem-ADM4union
BRA-Belem-ADM4
BRA-Belem-ADM2union
BRA-Belem-ADM2
CRI-San_Jose-ADM2union
CRI-San_Jose-ADM2
RWA-Kigali-ADM4union
RWA-Kigali-ADM4
SLE-Freetown-ADM4cityunion
SLE-Freetown-ADM4city
SLE-Freetown-ADM4regionunion
SLE-Freetown-ADM4region
MAR-Marrakech-ADM2
MAR-Marrakech-ADM5
IND-Chennai-ADM4union
IND-Chennai-ADM4
IND-Chennai-ADM6union
IND-Chennai-ADM6
IND-Pune-ADM4union
IND-Pune-ADM4
IND-Surat-ADM4union
IND-Surat-ADM4
CHN-Chengdu-ADM3union
CHN-Chengdu-ADM3
CHN-Chongqing-ADM1
CHN-Chongqing-ADM3
CHN-Ningbo-ADM3union
CHN-Ningbo-ADM3
IDN-Jakarta-ADM4union
IDN-Jakarta-ADM4
IDN